<img src="https://s3.amazonaws.com/datascienceheroes.com/EDV/360_banner_python.png" width="400px">


# Escuela de Datos Vivos 

## LAB P.D.3.1-3.2: Introducción al one hot encoding / dummy vars
Creado por Pablo Casas | https://escueladedatosvivos.ai   

In [1]:
import pandas as pd
from qgrid import show_grid
import seaborn as sns
from pandas_profiling import ProfileReport

### Puntos importantes sobre one hot encoding / dummy

a) Tiene que tener las variables originales

b) Tiene que manejar las categorias nuevas

c) Queremos que corra para todas las variables categóricas juntas, elimine las orignales y deje las numéricas.

Mención: `OneHotEncoding` de `sklearn` vs `get_dummies` de `pandas`:


## 1) Creamos datos de ejemplo

In [2]:
d_tr = pd.DataFrame({"auto":["Audi", "BMW", "Mazda"], "costo":[100,50,120], "color":["verde", "verde", "azul"]})

In [3]:
d_tr

,auto,costo,color
0,Audi,100,verde
1,BMW,50,verde
2,Mazda,120,azul


## 2) Ejecutamos `get_dummies`

In [4]:
d_tr_ohe = pd.get_dummies(d_tr) 

# nota: vamos a olvidarnos de los nulos

In [5]:
d_tr_ohe

,costo,auto_Audi,auto_BMW,auto_Mazda,color_azul,color_verde
0,100,1,0,0,0,1
1,50,0,1,0,0,1
2,120,0,0,1,1,0


## 3) Importante: guardar el encoding original

In [6]:
d_tr_ohe.columns

Index(['costo', 'auto_Audi', 'auto_BMW', 'auto_Mazda', 'color_azul',
       'color_verde'],
      dtype='object')

Con `pickle`  guardamos para aplicar en producción con datos nuevos/nunca vistos

Guardamos:

In [7]:
import pickle

with open('categories_ohe.pickle', 'wb') as handle:
    pickle.dump(d_tr_ohe.columns, handle, protocol=pickle.HIGHEST_PROTOCOL)

## *) Acá hacemos nuestra magia... 

![](https://miro.medium.com/max/1313/1*CUae0iZD_ejTLqFnbuzJqw.gif)

#### Deployamos el proyecto

#### Lo ponemos en producción

#### Necesitamos las transformaciones originales



#### Simulamos un data set nuevo (con datos que ántes no vio):

In [8]:
d_ts = pd.DataFrame({"auto":["Audi", "Ford"], "costo":[100,30], "color":["verde", "negro"]})
d_ts

,auto,costo,color
0,Audi,100,verde
1,Ford,30,negro


Nota! Hay valores nuevos: 
    
- color: `negro`
- auto: `Ford`


---

## "4") Cargamos el encoding utilizado

Encoding o transformación

Y acá lo levantamos para reutilizar `N` veces (estamos simulando que lo estamos usando en productivo):

In [9]:
with open('categories_ohe.pickle', 'rb') as handle:
    ohe_tr = pickle.load(handle)

In [10]:
ohe_tr

Index(['costo', 'auto_Audi', 'auto_BMW', 'auto_Mazda', 'color_azul',
       'color_verde'],
      dtype='object')

## 5) Hacemos get_dummies al TS con el encoding del TR

In [11]:
pd.get_dummies(d_ts).reindex(columns = ohe_tr)

,costo,auto_Audi,auto_BMW,auto_Mazda,color_azul,color_verde
0,100,1,NaN,NaN,NaN,1
1,30,0,NaN,NaN,NaN,0


¿Que pasó acá?

## 6) Remplazar categorías faltantes en `get_dummies` 

In [12]:
pd.get_dummies(d_ts).reindex(columns = ohe_tr).fillna(0)

,costo,auto_Audi,auto_BMW,auto_Mazda,color_azul,color_verde
0,100,1,0.0,0.0,0.0,1
1,30,0,0.0,0.0,0.0,0


`fillna` nos deja un tipo de dato mixto, así que una alternativa es:

In [13]:
pd.get_dummies(d_ts).reindex(columns = ohe_tr, fill_value=0)

,costo,auto_Audi,auto_BMW,auto_Mazda,color_azul,color_verde
0,100,1,0,0,0,1
1,30,0,0,0,0,0


### Prestemos atención a lo siguiente:

a) Valores nuevos: auto Ford y color negro

In [14]:
d_ts 

,auto,costo,color
0,Audi,100,verde
1,Ford,30,negro


In [15]:
d_tr # Este es el `d_tr` original:

,auto,costo,color
0,Audi,100,verde
1,BMW,50,verde
2,Mazda,120,azul


b) En el 2do registro, no existe ninguna categoría que existia en el TR (por eso lo dejó en 0)

**No rompió el flujo. Aunque quizás debería, todo depende.**

## 7) Cómo usar `get_dummies`

Evitar la correlación de variables redundantes.

In [16]:
d_tr_ohe_2 = pd.get_dummies(d_tr, drop_first=True, dummy_na=True) # 1- eliminen variables redundantes
    
d_tr_ohe_2.columns                                                # 2-guarden las categorías 

Index(['costo', 'auto_BMW', 'auto_Mazda', 'auto_nan', 'color_verde',
       'color_nan'],
      dtype='object')

---

## Resumen!




- Al usar dummy / one hot encoding para poner un modelo en producción, tener la precaución de los valores nuevos
- Si se usa `get_dummies` guardar el `.columns`, y al levantar el encoding usar el `reindex`.

---

## Ejercicios!

Basandose en el lab de P.C.2:

1) Cargar los datos de 'data/hotels_smp.csv' (fuente: [tidytuesday](https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-02-11/readme.md))

2) Separar en training (TR) y testing (TS).

3) Discretizar las variables categóricas usando TR (`get_dummies`), y aplicar la discretización en TS. Dentro del TS, modifiquen "a mano" una de las variables categóricas para que haya un valor nuevo. 

4) Parte fundamental del data scientist es inspeccionar que las transformaciones, y los flujos de datos sean los esperados. Por tal motivo debe auditarse el resultado: Revisar que la transformación sea la correcta. 